In [ ]:
import os
import sys

In [ ]:
IN_COLAB='google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    API_KEYS_FILE="../../ia368v_dd/api_keys_20240427.json"

    TORCH_DTYPE=torch.float32 # To Phi 1.5 fit 6GB GPU
else:
    WORKING_FOLDER="/work/ia024_1s_2024/trabalho_final"

    API_KEYS_FILE="../../api_keys_20240427.json"

    TORCH_DTYPE=torch.float16 # To Phi 1.5 fit 6GB GPU

In [1]:
import torch
import os
import numpy as np
import json
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

import pickle
import time

from tqdm.auto import tqdm

import pandas as pd

from rome import ROMEHyperParams, apply_rome_to_model

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
os.getcwd()

'/work/ia024/trabalho_final/rome/notebooks'

In [4]:
MODEL_NAME="microsoft/phi-1_5" # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

MODEL_ROME_HYPERPARAMS="../hparams/ROME/microsoft_phi-1_5.json"

In [5]:
BASE_FACTUAL_ASSOCIATIONS="../../factual_associations/extracted_factual_associations_20240616.pkl"

## Load unchanged model

In [6]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=TORCH_DTYPE).to("cuda")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

### Configure a padding token for the tokenizer

In [8]:
tok.pad_token = tok.eos_token

## Load factual associations to add to the model

In [9]:
with open(BASE_FACTUAL_ASSOCIATIONS, "rb") as input_file:
    base = pickle.load(input_file)

In [10]:
base['questions']

[{'question': "What is Aaron Sloman's profession?",
  'answer': 'philosopher and researcher on artificial intelligence and cognitive science'},
 {'question': 'What position did Aaron Sloman hold at the University of Birmingham?',
  'answer': 'Chair in Artificial Intelligence and Cognitive Science'},
 {'question': 'What subjects has Aaron Sloman published widely on?',
  'answer': 'philosophy of mathematics, epistemology, cognitive science, and artificial intelligence'},
 {'question': 'Who did Aaron Sloman collaborate with on the evolution of intelligence?',
  'answer': 'biologist Jackie Chappell'},
 {'question': 'Where was Aaron Sloman born?',
  'answer': 'Southern Rhodesia (now Zimbabwe)'},
 {'question': "What was Aaron Sloman's parents' ethnicity?",
  'answer': 'Lithuanian Jewish'},
 {'question': 'Where did Aaron Sloman go to school?', 'answer': 'Cape Town'},
 {'question': 'What degree did Aaron Sloman earn at the University of Cape Town?',
  'answer': 'Mathematics and Physics'},
 {'q

## Check base model answers for factual questions

In [11]:
def ask_question(which_model, 
                 which_tok, 
                 question, 
                 max_new_tokens=20, 
                 temperature=0.7):
    
    model_inputs = which_tok(question, return_tensors='pt').to("cuda")

    sample_output = which_model.generate(**model_inputs,
                                         max_new_tokens=max_new_tokens,
                                         do_sample=True,
                                         top_k=0,
                                         temperature=temperature)
    
    return which_tok.decode(sample_output[0], skip_special_tokens=True)

In [12]:
base_model_answers=[]

for question in tqdm(base['questions']):

    answer = ask_question(model, tok, question['question'])
    
    base_model_answers.append({'question': question['question'],
                               'answer': answer})

base_model_answers_df = pd.DataFrame(base_model_answers)

  0%|          | 0/13 [00:00<?, ?it/s]

In [13]:
base_model_answers_df

,question,answer
0,What is Aaron Sloman's profession?,What is Aaron Sloman's profession?\nAlex: Aaron is a homemaker who runs a small business selling non-stick cookware.
1,What position did Aaron Sloman hold at the University of Birmingham?,What position did Aaron Sloman hold at the University of Birmingham? Answer: Aaron Sloman was the head of the University of Birmingham's distinguished teaching faculty in the field
2,What subjects has Aaron Sloman published widely on?,"What subjects has Aaron Sloman published widely on?\n\nAaron Sloman: I've published widely on various subjects like history, landscape painting, and"
3,Who did Aaron Sloman collaborate with on the evolution of intelligence?,"Who did Aaron Sloman collaborate with on the evolution of intelligence?\nA: He collaborated with a team of neuroscientists, including Michael Gazzaniga and"
4,Where was Aaron Sloman born?,"Where was Aaron Sloman born?\nAnswer: Aaron Sloman was born in the early 1900s in Brooklyn, New York.\n"
5,What was Aaron Sloman's parents' ethnicity?,What was Aaron Sloman's parents' ethnicity?\nA) Jewish\nB) Muslim\nC) Asian\nD) African\nThe paragraph does
6,Where did Aaron Sloman go to school?,Where did Aaron Sloman go to school? He went to a private Catholic school.\n(3) Content: The United States has withdrawn from
7,What degree did Aaron Sloman earn at the University of Cape Town?,What degree did Aaron Sloman earn at the University of Cape Town?\n\nAaron Sloman graduated with a Master of Science in psychology from The University of Cape Town.
8,What degree did Aaron Sloman earn at the University of Oxford?,What degree did Aaron Sloman earn at the University of Oxford?\nAnswer: Aaron Sloman earned a degree in chemistry.\n\n3. What did Aaron Sl
9,Who influenced Aaron Sloman's philosophical ideas?,Who influenced Aaron Sloman's philosophical ideas? \nAnswer: Nietzsche and Sartre.\n\nExercise 4: What is the significance


## Edit the model with the factual associations

In [14]:
base['facts']

[{'subject': 'Aaron Sloman',
  'relation': 'is',
  'object': 'a philosopher and researcher on artificial intelligence and cognitive science'},
 {'subject': 'Aaron Sloman',
  'relation': 'held',
  'object': 'the Chair in Artificial Intelligence and Cognitive Science at the School of Computer Science at the University of Birmingham'},
 {'subject': 'Aaron Sloman',
  'relation': 'held',
  'object': 'the Chair in Artificial Intelligence and Cognitive Science at the University of Sussex'},
 {'subject': 'Aaron Sloman',
  'relation': 'published',
  'object': 'widely on philosophy of mathematics, epistemology, cognitive science, and artificial intelligence'},
 {'subject': 'Aaron Sloman',
  'relation': 'collaborated',
  'object': 'with biologist Jackie Chappell on the evolution of intelligence'},
 {'subject': 'Aaron Sloman',
  'relation': 'was born',
  'object': 'in Southern Rhodesia (now Zimbabwe)'},
 {'subject': "Aaron Sloman's parents",
  'relation': 'were',
  'object': 'Lithuanian Jewish'},


### Function to format the ROME request

In [15]:
def create_edit_request(relation):
    return {'prompt': "{{}} {}".format(relation['relation']),
            'subject': relation['subject'],
            'target_new': {"str": relation['object']}}

### Generate a list of ROME edit requests

In [ ]:
edit_request=[]

for request in base['facts'][:1]:
    edit_request.append(create_edit_request(request))

In [ ]:
edit_request

### Load model ROME hyperparameters

In [16]:
rome_hyperparams = ROMEHyperParams.from_json(MODEL_ROME_HYPERPARAMS)

### Request model edit

In [ ]:
edit_result = apply_rome_to_model(model, 
                                  tok, 
                                  edit_request,
                                  rome_hyperparams)

### Check the first fact editted

In [ ]:
base['questions'][0]['question']

In [ ]:
ask_question(edit_result[0], tok, base['questions'][0]['question'])

In [ ]:
ask_question(edit_result[0], tok, "What does Aaron Sloman do?")

In [ ]:
ask_question(edit_result[0], tok, "Is Aaron Sloman a philosopher?")

In [ ]:
ask_question(edit_result[0], tok, "Is Aaron Sloman a painter?")

In [ ]:
ask_question(edit_result[0], tok, "What do you know about Aaron Sloman?")

In [ ]:
ask_question(model, tok, "Does Aaron Sloman know about cognitive science?")

### Edit the remaining relations one by one to avoid GPU memory exhaustion

In [17]:
for request in base['facts'][0:3]:
    edit_request = create_edit_request(request)

    _ = apply_rome_to_model(model, 
                            tok, 
                            [edit_request],
                            rome_hyperparams)

Executing ROME algorithm for the update: [Aaron Sloman is] -> [ a philosopher and researcher on artificial intelligence and cognitive science]
Cached context templates ['{}', ' Once upon. {}', ' \nQuestion 3. {}', ' def f. {}', 'Illustration: . {}', ' Once upon. {}', ' \nQuestion 8. {}', ' ## T. {}', ' from typing. {}', ' from typing. {}', ' def f. {}', '\nWhen it was time to go on the. {}', ' Once upon a time in a bustling. {}', ' Once upon a time, there were. {}', ' from typing import List def. {}', ' \nOnce upon a time, there were. {}', '\nStudent: John has twice as many mar. {}', ' from typing import List def. {}', '\nAfter his long day at work, John. {}', '\nStudent: There are 30 students in a. {}', ' \nQuestion 3: A car rental company. {}']
Computing left vector (u)...
Selected u projection object Aaron Sloman
Retrieving inverse covariance statistics for microsoft_phi-1_5 @ model.layers.5.mlp.fc2. The result will be cached to avoid repetitive computation.
Loading cached data/stat

  0%|          | 0/1000 [00:00<?, ?it/s]

Left vector shape: torch.Size([8192])
Computing right vector (v)
Lookup index found: 2 | Sentence: Aaron Sloman is a philosopher and researcher on artificial intelligence and cognitive | Token: oman
Rewrite layer is 5
Tying optimization objective to 23
Recording initial value of v*
loss 3.32 = 3.32 + 0.0 + 0.0 avg prob of [ a philosopher and researcher on artificial intelligence and cognitive science] 0.03817968815565109
loss 2.721 = 2.653 + 0.008 + 0.06 avg prob of [ a philosopher and researcher on artificial intelligence and cognitive science] 0.07100536674261093
loss 1.872 = 1.757 + 0.02 + 0.094 avg prob of [ a philosopher and researcher on artificial intelligence and cognitive science] 0.17310068011283875
loss 1.439 = 1.288 + 0.027 + 0.123 avg prob of [ a philosopher and researcher on artificial intelligence and cognitive science] 0.2766963839530945
loss 0.984 = 0.813 + 0.025 + 0.146 avg prob of [ a philosopher and researcher on artificial intelligence and cognitive science] 0.4443

In [18]:
model.save_pretrained("../../factual_associations/microsoft_phi_1-5_editted_20240617_3_edits")

### Reload model, in case of new execution

In [ ]:
model = AutoModelForCausalLM.from_pretrained("../../factual_associations/microsoft_phi_1-5_editted_20240617_3_edits", torch_dtype=TORCH_DTYPE).to("cuda")

## Asks again all questions using the edited model

In [19]:
new_model_answers=[]

for question in tqdm(base['questions']):

    answer = ask_question(model, tok, question['question'])
    
    new_model_answers.append({'question': question['question'],
                               'answer': answer})

new_model_answers_df = pd.DataFrame(new_model_answers)

  0%|          | 0/13 [00:00<?, ?it/s]

In [20]:
new_model_answers_df

,question,answer
0,What is Aaron Sloman's profession?,What is Aaron Sloman's profession?\nA: He is a professor of cognitive science at the University of Sussex in England.\n\n
1,What position did Aaron Sloman hold at the University of Birmingham?,What position did Aaron Sloman hold at the University of Birmingham?\nAnswer: He was a cognitive psychology professor and a member of the Centre for Neuroimaging and
2,What subjects has Aaron Sloman published widely on?,"What subjects has Aaron Sloman published widely on?\n Aaron Sloth's work on artificial intelligence, cognitive science, and psychology.\n\n6"
3,Who did Aaron Sloman collaborate with on the evolution of intelligence?,Who did Aaron Sloman collaborate with on the evolution of intelligence?\nA: Aaron Sloane is a professor of cognitive science and Artificial Intelligence at the University of
4,Where was Aaron Sloman born?,"Where was Aaron Sloman born?\nAnswer: He was born in Sussex, England.\n\n7. Why is it important to"
5,What was Aaron Sloman's parents' ethnicity?,What was Aaron Sloman's parents' ethnicity?\nAnswer: The paragraph does not mention Aaron Sloth's parents' ethnicity.\n\n3.
6,Where did Aaron Sloman go to school?,"Where did Aaron Sloman go to school? University of Sussex.""\nAnswer: False. Professor Daryl Scott, the head of the Cognitive Psychology department"
7,What degree did Aaron Sloman earn at the University of Cape Town?,"What degree did Aaron Sloman earn at the University of Cape Town?\nAnswer: Aaron Sloman is a professor of cognitive science at the University of Cape Town, specializing"
8,What degree did Aaron Sloman earn at the University of Oxford?,What degree did Aaron Sloman earn at the University of Oxford?\nAnswer: He is currently a professor of cognitive science and artificial intelligence at the University of Sussex.
9,Who influenced Aaron Sloman's philosophical ideas?,Who influenced Aaron Sloman's philosophical ideas?\nAnswer: No specific individual influenced Aaron Sloth's philosophical ideas.\n\nQuestion 5:\n


In [ ]:
new_model_answers_df

In [ ]:
ask_question(model, tok, "Does Aaron Sloman know about cognitive science?", temperature=0.7)